In [ ]:
import time
import datetime
import pandas as pd
import numpy as np
from collections import OrderedDict

#Analysis Data
#ETF: QYLD, NUSI, RYLD
#Stocks:AAPL, MS, MSFT
ticker = 'QYLD'
start_date = '2021-01-01'
end_date = '2021-10-25'

#Setup
target_dates = pd.date_range(start_date, end_date,freq='MS')#.strftime('%Y-%m-%d').tolist()
monthly_investment=100

period1 = int(time.mktime(datetime.datetime(2000, 1, 1, 23, 59).timetuple()))
period2 = int(time.mktime(datetime.datetime.now().timetuple()))
interval = '1d' # 1d, 1m

holdings_df = pd.DataFrame(columns=["Date","Share Price","Shares Purchased", "Dividend", "Dividend Shares", "Dividend Payout"])
dividends_df = pd.DataFrame(columns=["Date","Share Price","Shares Purchased", "Dividend", "Dividend Shares", "Dividend Payout"])
dividends = list()

def record_dividend(date, dividend):
    entry = {"Date": date,
             "Dividend" : dividend,
             "Dividend Shares": holdings_df[:date]['Shares Purchased'].sum(),
             "Dividend Payout" : (holdings_df[:date]['Shares Purchased'].sum())*dividend
            }
    return entry

#Get date from feed(s)
stock_query_string = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'
stock_df = pd.read_csv(stock_query_string)

div_query_string = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=div&includeAdjustedClose=true'
div_df = pd.read_csv(div_query_string, parse_dates=['Date'])
div_df.set_index('Date', inplace=True)
div_df.sort_index(inplace=True)


#Calculate Holdings for begining of month
bus_days = map(lambda x: x + pd.offsets.BDay(), target_dates)
bus_day_list = list(bus_days)
close_prices = []
for day in bus_day_list:
    carr = stock_df.loc[stock_df.Date==day.date().strftime('%Y-%m-%d')].Close.values
    if carr.size == 0:
        next_day = day - pd.offsets.BDay() - pd.offsets.BDay()
        carr = stock_df.loc[stock_df.Date==day.date().strftime('%Y-%m-%d')].Close.values

    if carr.size!=0:
        close = carr[0]
        
    close_prices.append((day.date().strftime('%Y-%m-%d'),close))
    entry = {'Date':day.date(), 'Share Price': close, 'Shares Purchased':monthly_investment/close}
    holdings_df = holdings_df.append(entry, ignore_index=True)
    
holdings_df['Date'] = pd.to_datetime(holdings_df['Date'])
holdings_df.set_index('Date', inplace=True)

#Calculate Dividends - if any - as they occour
divs_to_consider_df = div_df.loc[start_date:end_date]
dividends = [record_dividend(i, dividend) for i, dividend in divs_to_consider_df.itertuples()]
dividends_df = dividends_df.append(dividends, ignore_index = True)
dividends_df['Date'] = pd.to_datetime(dividends_df['Date'])
dividends_df.set_index('Date', inplace=True)

holdings = pd.concat([holdings_df,dividends_df])

total_shares = 0
for price in close_prices:
    monthly_shares = monthly_investment/price[1]
    total_shares = total_shares + monthly_shares
    
value = total_shares * close_prices[-1][1]
print("would spend ", len(close_prices)*monthly_investment, " over ", len(close_prices), "months")
print("and would have: ", total_shares, " shares, worth ", value, " at ", close_prices[-1][1])
print("Dividends erned: ",holdings['Dividend Payout'].sum())
holdings.replace(np.nan,'').sort_index()
